In [1]:
import os 


In [2]:
CUSTOM_MODEL_NAME = "objects" 
PRETRAINED_MODEL_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
PRETRAINED_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
TF_RECORD_SCRIPT_NAME = "generate_tfrecord.py"
LABEL_MAP_NAME = "label_map.pbtxt"


In [3]:
paths = {
    "WORKSPACE_PATH": os.path.join("Tensorflow", "workspace"),
    "SCRIPTS_PATH": os.path.join("Tensorflow","scripts"),
    "APIMODEL_PATH": os.path.join("Tensorflow","models"),
    "ANNOTATION_PATH": os.path.join("Tensorflow", "workspace","annotations"),
    "IMAGE_PATH": os.path.join("Tensorflow", "workspace","images"),
    "MODEL_PATH": os.path.join("Tensorflow", "workspace","models"),
    "PRETRAINED_MODEL_PATH": os.path.join("Tensorflow", "workspace","pre-trained-models"),
    "CHECKPOINT_PATH": os.path.join("Tensorflow", "workspace","models",CUSTOM_MODEL_NAME), 
    "OUTPUT_PATH": os.path.join("Tensorflow", "workspace","models",CUSTOM_MODEL_NAME, "export"), 
    "TFJS_PATH":os.path.join("Tensorflow", "workspace","models",CUSTOM_MODEL_NAME, "tfjsexport"), 
    "TFLITE_PATH":os.path.join("Tensorflow", "workspace","models",CUSTOM_MODEL_NAME, "tfliteexport"), 
    "PROTOC_PATH":os.path.join("Tensorflow","protoc")
 }

In [4]:
files = {
    "PIPELINE_CONFIG":os.path.join("Tensorflow", "workspace","models", CUSTOM_MODEL_NAME, "pipeline.config"),
    "TF_RECORD_SCRIPT": os.path.join(paths["SCRIPTS_PATH"], TF_RECORD_SCRIPT_NAME), 
    "LABELMAP": os.path.join(paths["ANNOTATION_PATH"], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        !mkdir -p {path}

### Install dependencies

In [25]:
#if not os.path.exists(os.path.join(paths["APIMODEL_PATH"], "research", "object_detection")):
!git clone https://github.com/tensorflow/models.git {paths["APIMODEL_PATH"]}
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py .

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 84605, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 84605 (delta 127), reused 219 (delta 115), pack-reused 84372
Receiving objects: 100% (84605/84605), 597.35 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (60616/60616), done.


In [27]:
VERIFICATION_SCRIPT = os.path.join(paths["APIMODEL_PATH"], "research", "object_detection", "builders", "model_builder_tf2_test.py")
# Verify Installation
!python3 {VERIFICATION_SCRIPT}

2023-04-09 00:19:49.490820: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-09 00:19:49.526249: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-09 00:19:49.526580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 00:19:50.312387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/snow/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modi

### set up model

In [36]:
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+".tar.gz"} {paths["PRETRAINED_MODEL_PATH"]}
!cd {paths["PRETRAINED_MODEL_PATH"]} && tar -zxvf {PRETRAINED_MODEL_NAME+".tar.gz"}

--2023-04-09 00:24:54--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2607:f8b0:4005:813::2010, 142.251.46.240
Connecting to download.tensorflow.org (download.tensorflow.org)|2607:f8b0:4005:813::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  14.7MB/s    in 1.3s    

2023-04-09 00:24:55 (14.7 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320

In [37]:
labels = [{"name":"mouse", "id":1}, {"name":"charger", "id":2}, {"name":"ball", "id":3}, {"name":"marker", "id":4}]

with open(files["LABELMAP"], "w") as f:
    for label in labels:
        f.write("item { \n")
        f.write("\tname:\"{}\"\n".format(label["name"]))
        f.write("\tid:{}\n".format(label["id"]))
        f.write("}\n")

In [38]:
!python3 {files["TF_RECORD_SCRIPT"]} -x {os.path.join(paths["IMAGE_PATH"], "train")} -l {files["LABELMAP"]} -o {os.path.join(paths["ANNOTATION_PATH"], "train.record")} 
!python3 {files["TF_RECORD_SCRIPT"]} -x {os.path.join(paths["IMAGE_PATH"], "test")} -l {files["LABELMAP"]} -o {os.path.join(paths["ANNOTATION_PATH"], "test.record")} 


Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [39]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [40]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2023-04-09 00:26:51.551048: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-09 00:26:51.586050: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-09 00:26:51.592705: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 00:26:52.327950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [42]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [43]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [44]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]


In [45]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [46]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [65]:
train = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=6000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

### Train

In [69]:
!{train}

2023-04-09 03:11:35.776241: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/snow/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
W0409 03:11:37.423997 139685467172864 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0409 03:11:37.435764 139685467172864 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/jo

### evaluation

In [67]:
evaluate = "python3 {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [68]:
!{evaluate}

2023-04-09 03:05:20.291347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/snow/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
W0409 03:05:21.787524 140124260134912 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0409 03:05:21.787629 140124260134912 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0

### Load model from checkpoint

In [6]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

2023-04-09 06:30:10.106549: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-09 06:30:10.139165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-09 06:30:10.139445: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 06:30:10.888295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/snow/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modi

In [7]:


# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-7')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections



### camera test!

In [8]:
import cv2 
import numpy as np

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [9]:
img = cv2.imread("Tensorflow/workspace/images/test/ball.433418f4-d65b-11ed-8421-2dc33355e2c0.jpg")
img_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(img_np, 0), dtype=tf.float32) 
detections = detect_fn(input_tensor)

print(detections)

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[1.3237786e-01, 4.6940529e-01, 7.2668958e-01, 9.0040386e-01],
        [1.4179063e-01, 4.6015599e-01, 7.2608316e-01, 8.9127493e-01],
        [4.2154598e-01, 4.4214803e-01, 7.2656524e-01, 9.0017432e-01],
        [1.3876677e-01, 4.6704900e-01, 7.1579087e-01, 9.0354586e-01],
        [1.4510870e-01, 2.6302218e-01, 7.7299869e-01, 8.4419560e-01],
        [5.3030872e-01, 4.8807740e-01, 7.3199856e-01, 8.9797378e-01],
        [0.0000000e+00, 0.0000000e+00, 6.9058788e-01, 8.2829225e-01],
        [1.2160055e-02, 6.3150048e-02, 2.4816877e-01, 5.0052112e-01],
        [5.2562070e-01, 4.7101161e-01, 7.2609901e-01, 9.0759778e-01],
        [1.5730879e-01, 4.6532330e-01, 7.2235143e-01, 8.9108717e-01],
        [9.0030804e-03, 7.5080389e-01, 1.8816432e-01, 9.9648052e-01],
        [1.4736009e-01, 4.8303014e-01, 4.5678902e-01, 8.6733860e-01],
        [1.4736009e-01, 4.8303014e-01, 4.5678902e-01, 8.6733860e-01],
        [4.215459

In [10]:
num_detections = int(detections.pop('num_detections'))
print(num_detections)
detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1


100


In [11]:
print(detections["detection_boxes"][0])
print(detections["detection_boxes"][0][0])

[0.13237786 0.4694053  0.7266896  0.90040386]
0.13237786


In [34]:
image_np_with_detections = img_np.copy()
    
viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
cv2.waitKey(0)

48

: 

In [99]:
video = cv2.VideoCapture(2)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

while video.isOpened(): 
    ret, frame = video.read()
    frame_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(frame_np, 0), dtype=tf.float32) 
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    label_id_offset = 1
    image_np_with_detections = frame_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        video.release()
        cv2.destroyAllWindows()
        break